In [ ]:
!pip install kaggle --upgrade --quiet

In [ ]:
import os
from getpass import getpass
import cv2

In [ ]:
# os.environ['KAGGLE_USERNAME'] = getpass('Enter KAGGLE_USERNAME secret value: ')
# os.environ['KAGGLE_KEY'] = getpass('Enter KAGGLE_KEY secret value: ')

Enter KAGGLE_USERNAME secret value: ··········
Enter KAGGLE_KEY secret value: ··········


In [ ]:
os.environ['KAGGLE_USERNAME'] = 'kyawlin'
os.environ['KAGGLE_KEY'] = '24d3269babdaae2289340829b8434315'

## Download Kaggle Test Dataset

In [ ]:
!kaggle datasets download -d kyawlin/football-player-detection
!unzip /content/football-player-detection.zip

100% 147M/147M [00:08<00:00, 22.2MB/s]
100% 147M/147M [00:08<00:00, 17.8MB/s]
Archive:  /content/football-player-detection.zip
  inflating: 20230212_last_model.pt  
  inflating: Faster-RCNN.txt         


In [ ]:
# # Extract all frames from video

# def extract_all(vid_pth,folder_name):
#   if not os.path.exists(folder_name):
#       os.makedirs(folder_name)

#   vidcap = cv2.VideoCapture(vid_pth)
#   success,image = vidcap.read()
#   count = 0
#   while success:
#     cv2.imwrite("all_frames/frame_{}.jpg".format(str(count).zfill(3)), image)     # save frame as JPEG file      
#     success,image = vidcap.read()
#     # print('Read a new frame: ', success)
#     count += 1
#   print('Done !')
#   print('No. of all frames : ',len(os.listdir('/content/all_frames')))

# extract_all(vid_pth,'all_frames')    # extract all frames from videos

## Import Dependencies

In [ ]:
#==================
# Import libraries
#==================
import gc
import cv2
import glob
import time
import copy
import random

import numpy as np 
import pandas as pd 
import os
from PIL import Image
from glob import glob
import xml.etree.ElementTree as ET 

# from omegaconf import OmegaConf
from pathlib import Path 
from typing import Union, Tuple, List, Iterable, Dict 
from collections import defaultdict

import logging 
from logging import  getLogger, StreamHandler, Formatter

import torch
import torchvision
from torchvision import transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import TensorDataset
from torch.optim import lr_scheduler 
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
from base64 import b64encode
from IPython.display import HTML


plt.style.use(['default']) 

from datetime import datetime
today = str(datetime.today()).split(" ")[0].replace("-", "")
print(today)

20230315


## Configs

In [ ]:
#==================
# Config
#==================
class CFG:
    # path
    model = "/content/20230212_last_model.pt"
    bdd_xml = ""
    bdd_img = ""
    test_path = "/content/frames"
    log_path = './'
    # class of dataset
    dataset_class = ['ball','goalkeeper','player','referee']
    # colors (RGB)
    colors = ((0,0,0),(222,49,99),(100,149,237),(255,191,0),(136,78,160))
    # hyper prams of Faster-RCNN
    epochs = 10
    batch_size = 1
    lr = 0.005 
    wd = 1e-6
    scheduler = 'CosineAnnealingLR' # ReduceLROnPlateau
    min_lr = 1e-6
    T_max = 2400
    T_0 = 25
    scale = 1080 # 720 # image_scale (Vertical)
    # hyper prams of Faster-RCNN inference
    threshold = 0.6
    # seed
    seed = 2022
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') 

cfg = CFG()

## Import Model

In [ ]:
#==================
# Seed
#==================
def set_seed(seed = 45):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    print('> SEEDING DONE')
    
set_seed(cfg.seed)

> SEEDING DONE


In [ ]:
#==================
# Logger
#==================
logger = getLogger(__name__)
logger.setLevel(logging.DEBUG)
filehandler = logging.FileHandler(os.path.join(cfg.log_path, "Faster-RCNN.txt"))
streamhander = StreamHandler()
filehandler.setLevel(logging.DEBUG)
streamhander.setLevel(logging.DEBUG)
handler_format = Formatter('[%(asctime)s][%(name)s][%(levelname)s][%(message)s]')
filehandler.setFormatter(handler_format)
streamhander.setFormatter(handler_format)
logger.addHandler(filehandler)
logger.addHandler(streamhander)

In [ ]:
# =====================
# Import Model
# =====================
data_class = cfg.dataset_class
data_class.insert(0, "__background__")
classes = tuple(data_class) 

model_inf = torchvision.models.detection.fasterrcnn_resnet50_fpn(
    weights=None, progress=False,
    num_classes=91, weights_backbone=None, trainable_backbone_layers=None
)
num_classes = len(cfg.dataset_class)
in_features = model_inf.roi_heads.box_predictor.cls_score.in_features
model_inf.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

model_inf.load_state_dict(torch.load(cfg.model))

<All keys matched successfully>

In [ ]:
import glob
import pylab
from tqdm import tqdm
# img_list = sorted(glob.glob(cfg.test_path+'/*.jpg'))

## Helper Functions

In [ ]:
def convert_frames2video(img_pth,vid_name):  # Detected frames into video
  img_array = []
  for filename in tqdm(sorted(glob.glob(img_pth+'/*.jpg'))):
      img = cv2.imread(filename)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      height, width, layers = img.shape
      size = (width,height)
      img_array.append(img)
      # print(filename, ' - ', size)

  out = cv2.VideoWriter(vid_name+'.avi',cv2.VideoWriter_fourcc(*'DIVX'), 25, size)
  
  for i in range(len(img_array)):
      out.write(cv2.cvtColor(img_array[i], cv2.COLOR_RGB2BGR))
  out.release()
  print('Converting completed. '+vid_name+'.avi created ...')

def convert_avi_to_mp4(avi_file_path, output_name):  # avi2mp4
    os.popen("ffmpeg -i '{input}' -ac 2 -b:v 2000k -c:a aac -c:v libx264 -b:a 160k -vprofile high -bf 0 -strict experimental -f mp4 '{output}.mp4'".format(input = avi_file_path, output = output_name))
    return True


## Detect video and generate output frames ##

def detect_video(output_,vid,device):
  if not os.path.exists(output_):
    os.mkdir(output_)
  model_inf.to(device)
  model_inf.eval()
  vidcap = cv2.VideoCapture(vid)
  success,image = vidcap.read()
  count = 0
  while success:
    # cv2.imwrite("all_frames/frame_{}.jpg".format(str(count).zfill(3)), image)     # save frame as JPEG file      
    # print('Read a new frame: ', success)
    # img = cv2.imread(image)
    success,image = vidcap.read()
    try :
      img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    except:
      pass;
    image_tensor = torchvision.transforms.functional.to_tensor(img)
    with torch.no_grad():
        prediction = model_inf([image_tensor.to(device)])
    
    for i,box in enumerate(prediction[0]['boxes']):
        score = prediction[0]['scores'][i].cpu().numpy()
        if score > cfg.threshold:
            score = round(float(score),2)
            cat = prediction[0]['labels'][i].cpu().numpy()
            txt = '{} {}'.format(classes[int(cat)], str(score))
            font = cv2.FONT_HERSHEY_SIMPLEX
            cat_size = cv2.getTextSize(txt, font, 0.5, 2)[0]
            c = cfg.colors[int(cat)]
            box=box.cpu().numpy().astype('int')
            cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), c , 2)
            cv2.rectangle(img,(box[0], box[1] - cat_size[1] - 2),(box[0] + cat_size[0], box[1] - 2), c, -1)
            cv2.putText(img, txt, (box[0], box[1] - 2), font, 0.5, (0, 0, 0), thickness=1, lineType=cv2.LINE_AA)
    cv2.imwrite(output_ + '/frame_'+str(count+1).zfill(3)+'.jpg',cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    print('frame '+ str(count)+' detected ...')
    count += 1 
  # plt.show()
  print("Completed !")

## Inference

In [ ]:
vid_pth = '/content/019d5b34_1.mp4'   # video path
op_pth = '/content/results'           # output frames path

In [ ]:
detect_video(op_pth,vid_pth,device) # Inference on video

In [ ]:
convert_frames2video(op_pth,'detect_output') # convert detected frames into video

convert_avi_to_mp4('detect_output.avi','detected') # avi to mp4

100%|██████████| 750/750 [00:24<00:00, 30.73it/s]


Converting completed...


In [ ]:
# Display the detected video
input_path = '/content/detect_output.avi' # Detected video path
compressed_path = "/content/compressed_output.mp4"  # Compressed video path

os.system(f"ffmpeg -i {input_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

## ByteTrack

In [ ]:
!git clone https://github.com/ifzhang/ByteTrack.git
!cd ByteTrack && pip3 install -r requirements.txt
!cd ByteTrack && python3 setup.py develop
!pip install cython_bbox

fatal: destination path 'ByteTrack' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl (15 kB)
  Using cached ninja-1.11.1-py2.py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (145 kB)
  Using cached lap-0.4.0-cp39-cp39-linux_x86_64.whl
  Using cached motmetrics-1.4.0-py3-none-any.whl (161 kB)
  Using cached filterpy-1.4.5-py3-none-any.whl
  Using cached onnx-1.8.1.tar.gz (5.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached onnxruntime-1.8.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.5 MB)
  Using cached onnx_simplifier-0.3.5-py3-none-any.whl
  Using cached onnxoptimizer-0.3.9-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (645 kB)
  Using cached xmltodict-

In [ ]:
import sys
sys.path.append("/content/ByteTrack")

In [ ]:
!pip install onemetric --quiet
!pip install loguru --quiet
!pip install thop --quiet
!pip install lap --quiet

In [ ]:
from dataclasses import dataclass

@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 30
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False

In [ ]:
from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch

In [ ]:
from typing import List

import numpy as np

"""
BYTETracker does not assign tracker_id to existing bounding boxes but rather
predicts the next bounding box position based on previous one. Therefore, we 
need to find a way to match our bounding boxes with predictions.

usage example:

byte_tracker = BYTETracker(BYTETrackerArgs())
for frame in frames:
    ...
    results = model(frame, size=1280)
    detections = Detection.from_results(
        pred=results.pred[0].cpu().numpy(), 
        names=model.names)
    ...
    tracks = byte_tracker.update(
        output_results=detections2boxes(detections=detections),
        img_info=frame.shape,
        img_size=frame.shape)
    detections = match_detections_with_tracks(detections=detections, tracks=tracks)
"""

# converts List[Detection] into format that can be consumed by match_detections_with_tracks function

# converts List[STrack] into format that can be consumed by match_detections_with_tracks function
def tracks2boxes(tracks: List[STrack]) -> np.ndarray:
    return np.array([
        track.tlbr
        for track
        in tracks
    ], dtype=float)


# matches our bounding boxes with predictions
def match_detections_with_tracks(prediction, tracks):
    detection_boxes = extract_boxes(prediction,with_score=False)
    tracks_boxes = tracks2boxes(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detection_boxes)
    track2detection = np.argmax(iou, axis=1)
    
    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            prediction[0]['tracker_id'][detection_index] = tracks[tracker_index].track_id
            # print(tracker_index, ' - ',detection_index)
    return prediction

In [ ]:
# def extract_players_boxes(prediction,with_score=True):
#   boxes = prediction[0]['boxes'].cpu().detach().numpy()
#   scores = prediction[0]['scores'].cpu().detach().numpy()
#   labels = prediction[0]['labels'].cpu().detach().numpy()

#   # Extract only players detections
#   boxes = boxes[np.where(labels == 3)]       
#   scores = scores[np.where(labels == 3)]

#   players_detections = list()
#   for b,s in zip(boxes,scores):
#     if with_score == False :
#       players_detections.append(([b[0],b[1],b[2],b[3]]))
#     else :
#       players_detections.append(([b[0],b[1],b[2],b[3],s]))
#   return np.array(players_detections)

In [ ]:
def extract_boxes(prediction,with_score=True):
  boxes = prediction[0]['boxes'].cpu().detach().numpy()
  scores = prediction[0]['scores'].cpu().detach().numpy()
  labels = prediction[0]['labels'].cpu().detach().numpy()

  # Extract only players detections
  # boxes = boxes[np.where(labels == 3)]       
  # scores = scores[np.where(labels == 3)]
  detections = list()
  for b,s in zip(boxes,scores):
    if with_score == False :
      detections.append(([b[0],b[1],b[2],b[3]]))
    else :
      detections.append(([b[0],b[1],b[2],b[3],s]))
  return np.array(detections)

In [46]:
# !rm -r /content/test
if not os.path.exists('test'):  # for detected frames
  os.mkdir('test')

In [ ]:
model_inf.to(device)
model_inf.eval()
vidcap = cv2.VideoCapture('/content/Project Name.mp4')
success,image = vidcap.read()
count = 0
# initiate tracker
byte_tracker = BYTETracker(BYTETrackerArgs())

while success:
  # cv2.imwrite("all_frames/frame_{}.jpg".format(str(count).zfill(3)), image)     # save frame as JPEG file      
  # print('Read a new frame: ', success)
  # img = cv2.imread(image)
  success,image = vidcap.read()
  try :
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  except:
    pass;
  image_tensor = torchvision.transforms.functional.to_tensor(img)
  with torch.no_grad():
      predictions = model_inf([image_tensor.to(device)])
      t = list()
      for b in predictions[0]['boxes']:
        t.append(None)
      predictions[0]['tracker_id'] = t

  tracks = byte_tracker.update(
    output_results=extract_boxes(predictions),
    img_info=img.shape,
    img_size=img.shape)
  tracked_detections = match_detections_with_tracks(prediction = predictions, tracks=tracks)
  # print(predictions)
  for i,box in enumerate(tracked_detections[0]['boxes']):
    score = tracked_detections[0]['scores'][i].cpu().numpy()
    if tracked_detections[0]['tracker_id'][i] == None : continue
    else : 
      if score > cfg.threshold:
        score = round(float(score),2)
        cat = tracked_detections[0]['labels'][i].cpu().numpy()
        txt = '{} {}'.format(classes[int(cat)], str(score))
        font = cv2.FONT_HERSHEY_SIMPLEX
        cat_size = cv2.getTextSize(txt, font, 0.5, 2)[0]
        c = cfg.colors[int(cat)]
        box=box.cpu().numpy().astype('int')
        cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), c , 2)
        cv2.rectangle(img,(box[0], box[1] - cat_size[1] - 2),(box[0] + cat_size[0], box[1] - 2), c, -1)
        cv2.putText(img, txt, (box[0], box[1] - 2), font, 0.5, (0, 0, 0), thickness=1, lineType=cv2.LINE_AA)
    # # plt.imshow(img)
  img_name = '/content/test' + '/test_'+str(count+1).zfill(4)+'.jpg'
  cv2.imwrite(img_name,cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
  count += 1 
  print('frame '+ str(count)+' detected ...')

In [ ]:
convert_frames2video('/content/test','tracked_output') 

In [ ]:
input_path = '/content/tracked_output.avi' # Detected video path
compressed_path = "/content/c.mp4"  # Compressed video path

In [ ]:
# Display the tracked detected video

os.system(f"ffmpeg -i {input_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Output hidden; open in https://colab.research.google.com to view.